<a href="https://colab.research.google.com/github/ArifAygun/CustomerEye/blob/main/Trustpilot_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import random
import pandas as pd

from pprint import pprint
from urllib.parse import urlparse

import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup

In [12]:
from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/My Drive/Trustpilot/


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Trustpilot


In [13]:
# Define a function for scraping Trustpilot reviews
def scrape_trustpilot_reviews(company_url, start_page, end_page):

    # Define the path to the Excel file
    excel_file_path = 'proxy_user_agent_list.xlsx'

    # Read the 'ProxyList' sheet for proxies
    df_proxy_list = pd.read_excel(excel_file_path, 'ProxyList')
    proxy_list = df_proxy_list.iloc[:, 0].tolist()
    random_proxies = {"http": random.choice(proxy_list)}

    # Read the 'UserAgentList' sheet for user agents
    df_ua_list = pd.read_excel(excel_file_path, 'UserAgentList')
    user_agent_list = df_ua_list.iloc[:, 0].tolist()
    random_user_agent = random.choice(user_agent_list)
    random_header = {"user-agent": random_user_agent}

    pprint(random_header)
    pprint(random_proxies)

    # Create empty lists to store the data
    reviewer_names = []
    reviews_count = []
    country_code = []
    experience_dates = []
    ratings = []
    review_dates = []
    review_titles = []
    review_texts = []
    reply_dates = []
    reply_texts = []

    # Record the start time
    start_time = time.time()

    try:
        # Loop to navigate through pages
        for page_number in range(start_page, end_page + 1):  # Specify the range

            # Record the start time for this page
            page_start_time = time.time()

            # Construct the URL with the desired page number
            url_page = f"{company_url}?page={page_number}"
            response = requests.get(url_page, headers=random_header, proxies=random_proxies)
            html = response.content
            soup = BeautifulSoup(html, "html.parser")

            # Extract the domain name and subdomain from the URL for the file name
            parsed_url = urlparse(url_page)
            subdomain = parsed_url.path.strip('review/www.').replace('review/www.', '').replace('.com', '')

            # Get the current date and time
            current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

            # Add the current date to the CSV file name
            file_name = f"{subdomain}_{current_date}.csv"

            # Generate a random sleep duration
            sleep_duration = random.uniform(3, 12)
            # Sleep for the random duration
            time.sleep(sleep_duration)


            ########## Extract reviewer names
            reviewer_name_links = soup.find_all('div', {'class': 'styles_consumerDetailsWrapper__p2wdr'})
            reviewer_names.extend([
                link.find('span', {'class': 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}).text.strip()
                if link.find('span', {'class': 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'})
                else ""
                for link in reviewer_name_links
            ])


            ########## Extract review titles
            review_title_links = soup.find_all('a', {'class': 'link_internal__7XN06 typography_appearance-default__AAY17 typography_color-inherit__TlgPO link_link__IZzHN link_notUnderlined__szqki'})
            review_titles.extend([
                link.find('h2', {'class': 'typography_heading-s__f7029 typography_appearance-default__AAY17'}).text.strip()
                if link.find('h2', {'class': 'typography_heading-s__f7029 typography_appearance-default__AAY17'})
                else ""
                for link in review_title_links
            ])


            ########## Extract review texts
            review_text_links = soup.find_all('div', {'class': 'styles_reviewContent__0Q2Tg'})
            review_texts.extend([
                link.find('p', {'class': 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'}).text.strip()
                if link.find('p', {'class': 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'})
                else ""
                for link in review_text_links
            ])


            ########## Extract review dates
            review_date_links = soup.find_all('div', {'class': 'styles_reviewHeader__iU9Px'})
            review_dates.extend([
                link.find('div', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH'}).text.strip()
                if link.find('div', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH'})
                else ""
                for link in review_date_links
            ])


            ########## Extract review replies
            review_reply_links = soup.find_all('div', {'class': 'styles_reviewCardInner__EwDq2'})
            reply_texts.extend([
                link.find('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX'}).text.strip()
                if link.find('p', {'class': 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 styles_message__shHhX'})
                else ""
                for link in review_reply_links
            ])


            ########## Extract review reply dates
            review_reply_dates_links = soup.find_all('div', {'class': 'styles_reviewCardInner__EwDq2'})
            for link in review_reply_dates_links:
                reply_info_element = link.find('div', {'class': 'styles_replyInfo__FYSje'})

                if reply_info_element:
                    reply_info = reply_info_element.text.strip()
                    reply_dates.append(reply_info)
                else:
                    reply_dates.append("")


            ########## Extract ratings
            # Find all the div elements with the specified class
            div_elements = soup.find_all('div', {'class': 'styles_reviewHeader__iU9Px'})

            # Loop through each div element
            for div_element in div_elements:

                # Find the img tag within the div element
                img_tag = div_element.find('img')
                if img_tag:
                    # Extract the src attribute
                    src_attribute = img_tag.get('src')

                    # Split the src attribute by '/' and get the last part
                    parts = src_attribute.split('/')
                    last_part = parts[-1]

                    # Extract the numeric part from the last part
                    numeric_rating = last_part.split('-')[1].split('.')[0]

                    # Convert the numeric rating to an integer and append to the ratings list
                    ratings.append(int(numeric_rating))


            ########## Extract date of experience
            # Find all elements with the specified class for date of experience
            elements = soup.find_all('p', class_='typography_body-m__xgxZ_ typography_appearance-default__AAY17')

            # Extract the dates and store them in a list
            for element in elements:
                text = element.get_text(strip=True)  # Get the text content
                if "Date of experience" in text:
                    date = text.split(':')[-1].strip()  # Extract the date part
                    experience_dates.append(date)


            ########## Extract number of reviews, country code
            # Find all the div elements with the specified class
            consumer_details_divs = soup.find_all('div', class_='styles_consumerExtraDetails__fxS4S')

            # Iterate through the div elements and extract the desired information
            for div in consumer_details_divs:
                # Extract the number of reviews
                reviews_count_element = div.find('span', {'data-consumer-reviews-count-typography': True})
                reviews_count.append(reviews_count_element.get_text(strip=True) if reviews_count_element else "")

                # Extract the country code
                country_code_element = div.find('div', {'data-consumer-country-typography': True})
                if country_code_element:
                    country_code_span = country_code_element.find('span')
                    country_code_text = country_code_span.get_text(strip=True) if country_code_span else ""
                    country_code.append(country_code_text)
                else:
                    country_code.append("")


            ########## Print times of process
            # Record the end time for this page
            page_end_time = time.time()

            # Calculate the time taken to scrape this page
            page_time_taken = page_end_time - page_start_time

            # Record the end time
            end_time = time.time()
            # Calculate the total scraping time
            total_time = end_time - start_time
            minutes, seconds = divmod(total_time, 60)

            # Print the waiting time
            print(f"Waiting for {sleep_duration:.2f} seconds before scraping page {page_number}...")
            # Print the current page being scraped
            print(f"Scraping page {page_number}...")
            # Print the time taken for this page
            print(f"Time taken to scrape page {page_number}: {page_time_taken:.2f} seconds")
            # Print the total time of scraping
            print(f"Total Scraping Time: {minutes} min {seconds:.2f} sec")


    finally:
        print("Scraping finished")

    # Create a DataFrame from the collected data
    df_out = pd.DataFrame({
        'Reviewer Name': reviewer_names,
        'Reviews Count': reviews_count,
        'Country Code': country_code,
        'Experience Date': experience_dates,
        'Rating': ratings,
        'Review Date': review_dates,
        'Review Title': review_titles,
        'Review Text': review_texts,
        'Reply Date': reply_dates,
        'Reply Text': reply_texts
    })


    # Specify the base directory path where you want to save the CSV files on Google Drive
    base_directory_path = '/content/drive/My Drive/Trustpilot'

    # Create a directory for each subdomain on Google Drive
    subdomain_directory_path = f"{base_directory_path}/{subdomain}"

    # Check if the directory already exists, if not, create it
    if not os.path.exists(subdomain_directory_path):
        os.makedirs(subdomain_directory_path)

    # Get the current date and time
    current_date = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    # Add the current date to the CSV file name
    file_name = f"{subdomain_directory_path}/{subdomain}_{current_date}.csv"

    # Save the DataFrame to a CSV file on Google Drive
    df_out.to_csv(file_name, index=False)
    print(f"Scraping complete. Data saved to '{file_name}' on Google Drive")


In [ ]:
company_url = "https://www.trustpilot.com/review/pdfsimpli.com"
start_page = 991 # Specify the start page
end_page = 3298  # Specify the end page


# Iterate over the page ranges increments
for i in range(start_page, end_page + 1, 110):
    # Get the current start and end pages
    current_start_page = i
    current_end_page = min(i + 109, end_page)

    # Scrape reviews for the current page range
    scrape_trustpilot_reviews(company_url, current_start_page, current_end_page)

{'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Chrome/38.0.2125.111 Safari/537.36'}
{'http': '203.28.9.89:80'}
Waiting for 10.81 seconds before scraping page 1...
Scraping page 1...
Time taken to scrape page 1: 11.67 seconds
Total Scraping Time: 0.0 min 11.67 sec
Waiting for 4.34 seconds before scraping page 2...
Scraping page 2...
Time taken to scrape page 2: 4.84 seconds
Total Scraping Time: 0.0 min 16.51 sec
Waiting for 3.49 seconds before scraping page 3...
Scraping page 3...
Time taken to scrape page 3: 4.38 seconds
Total Scraping Time: 0.0 min 20.90 sec
Waiting for 11.43 seconds before scraping page 4...
Scraping page 4...
Time taken to scrape page 4: 12.17 seconds
Total Scraping Time: 0.0 min 33.07 sec
Waiting for 6.20 seconds before scraping page 5...
Scraping page 5...
Time taken to scrape page 5: 6.92 seconds
Total Scraping Time: 0.0 min 39.99 sec
Waiting for 5.73 seconds before scraping page 6...
Scraping page 6..